In [3]:
from PIL import Image
import pandas as pd
import json
import os
from os.path import join
import json
import torch
from transformers import CLIPProcessor, CLIPModel

In [4]:
root = "../images/web"
img_folders = os.listdir(root)

img_path = []
for folder in img_folders:
    img_names = os.listdir(join(root, folder))
    for name in img_names:
        img_path.append(join(root, folder, name))
categories = [p.split("\\")[-2] for p in img_path]
details = [p.split("\\")[-1].split(".")[0][:-1] for p in img_path]

df = pd.DataFrame(dict(category=categories, detail=details, img_path=img_path))

img_pairs_list = []
for i in range(len(df)):
    anchor = df.img_path[i]
    img_pairs = [dict(anchor=anchor, pair=pair) for pair in df.img_path.values if anchor != pair]
    img_pairs_list.append(img_pairs)
    
df["img_pairs"] = img_pairs_list

In [5]:
image_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16").image_processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").vision_model
model.eval()
,

''

In [6]:
images = []
for i in range(len(df)):
    x = Image.open(df.img_path[i]).convert("RGB")
    x = image_processor(x, return_tensors="pt").pixel_values[0]
    images.append(x[None,:])
images = torch.cat(images, dim=0)

In [7]:
embeds = model.forward(images).last_hidden_state[:,0,:]
embeds = embeds / embeds.norm(dim=-1, keepdim=True)

sim = embeds @ embeds.T
sim_ = sim.view(-1)[sim.view(-1) < 0.999].view(sim.size(0), -1).tolist()

In [8]:
img_pairs = list(df.img_pairs.values)
df.img_pairs = [[dict(anchor=pair_dict["anchor"], pair=pair_dict["pair"], score=str(score)) for pair_dict, score in zip(img_pairs[i], sim_[i])] for i in range(len(img_pairs))]

In [9]:
img_pairs = list(df.img_pairs.values)
img_pairs = sum(img_pairs, [])

In [10]:
df_img_pairs = pd.DataFrame(dict(data=img_pairs))
df_img_pairs["temp"] = df_img_pairs.data.apply(lambda x: " ".join(sorted(list(x.values()))))
df_img_pairs = df_img_pairs.drop_duplicates(subset=["temp"]).reset_index(drop=True)
df_img_pairs = df_img_pairs.drop("temp", axis=1)

df_img_pairs

,data
0,"{'anchor': '../images/web\cat\persian1.jpg', '..."
1,"{'anchor': '../images/web\cat\persian1.jpg', '..."
2,"{'anchor': '../images/web\cat\persian1.jpg', '..."
3,"{'anchor': '../images/web\cat\persian1.jpg', '..."
4,"{'anchor': '../images/web\cat\persian1.jpg', '..."
...,...
185,"{'anchor': '../images/web\vehicles\car1.jpg', ..."
186,"{'anchor': '../images/web\vehicles\car1.jpg', ..."
187,"{'anchor': '../images/web\vehicles\car2.jpg', ..."
188,"{'anchor': '../images/web\vehicles\car2.jpg', ..."


In [11]:
img_pairs = list(df_img_pairs.data.values)

In [11]:
img_pairs_tags = [[d["anchor"].split("\\")[-1][:-4], d["pair"].split("\\")[-1][:-4], d["score"]] for d in img_pairs]

In [15]:
img_pairs_ = [dict(anchor=p["anchor"][1:].replace("\\","/"), pair=p["pair"][1:].replace("\\", "/"), score=float(p["score"])) for p in img_pairs]

with open("image_pairs.json", "w") as f:
    json.dump(img_pairs_, f)